# Agentic AI: CSTR Reactor Safety System

### 🔬 Chemical Engineering meets Artificial Intelligence

This notebook demonstrates a **Physics-Informed Agentic System**. It combines classical Machine Learning with Generative AI to manage a Continuous Stirred-Tank Reactor (CSTR).

**Key Components:**

1. **Fault Detection:** A Random Forest model trained on synthetic reactor data.
2. **Reasoning Agent:** Google Gemini 1.5 Flash interpreting Standard Operating Procedures (SOPs).
3. **Safety Interlock:** A hard-coded deterministic layer that overrides AI suggestions if they violate physical safety constraints.

---


In [2]:
# Install required libraries
!pip install -q -U google-generativeai gradio pandas scikit-learn

---

## 1. Physics-Informed ML Training

In this section, we generate synthetic data representing reactor temperatures () and pressures (). We train a `RandomForestClassifier` to identify the "Danger Zone."

---



In [3]:
import gradio as gr
import pandas as pd
import numpy as np
import google.generativeai as genai
from sklearn.ensemble import RandomForestClassifier
import os

def train_model():
    # Synthetic Data Generation
    np.random.seed(42)
    temp = np.random.uniform(300, 500, 2000)
    press = np.random.uniform(50, 250, 2000)
    flow = np.random.uniform(10, 100, 2000)

    # Logic: Fault if Temp > 380 OR Pressure > 140
    target = ((temp > 380) | (press > 140)).astype(int)

    df = pd.DataFrame({'temp': temp, 'press': press, 'flow': flow, 'target': target})
    model = RandomForestClassifier(n_estimators=100)
    model.fit(df[['temp', 'press', 'flow']], df['target'])
    return model

clf = train_model()
print("✅ Model trained successfully on synthetic reactor data.")

/usr/local/lib/python3.12/dist-packages/google/colab/_import_hooks/_hook_injector.py:55: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as possible.
See README for more details:

https://github.com/google-gemini/deprecated-generative-ai-python/blob/main/README.md

  loader.exec_module(module)


✅ Model trained successfully on synthetic reactor data.


---

## 2. AI Setup & Knowledge Base

We configure the Gemini API and define the **Standard Operating Procedures (SOPs)** that the AI agent will use to make decisions.

> **Note:** Replace the placeholder API key with your own from Google AI Studio.

---



In [4]:
# SETUP CONFIGURATION
# Best practice: Use Colab Secrets (Left sidebar > Key icon) to store API_KEY
from google.colab import userdata
try:
    API_KEY = userdata.get('GEMINI_API_KEY')
except:
    API_KEY = "YOUR_KEY_HERE" # Fallback

genai.configure(api_key=API_KEY)
model_gemini = genai.GenerativeModel('gemini-1.5-flash')

sops = {
    "High_Heat_Fault": "CRITICAL: Exothermic runaway suspected. ACTION: Increase coolant flow to max. If pressure > 150psi, open emergency vent V-301.",
    "Normal_Ops": "System stable. Maintain current setpoints."
}

---

## 3. Core Logic & Safety Interlock

This is the heart of the "Agentic" system.

* **Safety Interlock:** If the AI hallucinates and suggests closing a vent during high pressure, this function forces a rejection.
* **Process Monitor:** Coordinates the ML prediction, Gemini reasoning, and safety check.

---

In [5]:
def safety_interlock(temp, press, ai_advice):
    """Deterministic safety guardrail that overrides AI responses"""
    if press > 150 and any(word in ai_advice.lower() for word in ["close", "shut"]) and "vent" in ai_advice.lower():
        return "⚠️ CRITICAL REJECTION: AI suggested closing vent during high pressure. Interlock forced VENT OPEN."
    return "✅ SAFETY CLEARED"

def process_monitor(temp, press, flow):
    # Step A: ML Prediction
    input_df = pd.DataFrame([[temp, press, flow]], columns=['temp', 'press', 'flow'])
    prediction = clf.predict(input_df)[0]

    # Manual Sensitivity Override for Demo
    if temp > 385 or press > 145:
        status = "High_Heat_Fault"
    else:
        status = "High_Heat_Fault" if prediction == 1 else "Normal_Ops"

    # Step B: Agentic Reasoning (Gemini)
    prompt = f"""
    You are a Lead Process Engineer.
    Current Sensors: Temp {temp}K, Pressure {press}psi, Flow {flow}L/min.
    ML Detection: {status}
    Official SOP: {sops[status]}

    Instruction: Give one clear, 1-sentence engineering command to the operator.
    """

    try:
        response = model_gemini.generate_content(prompt)
        ai_response = response.text
    except Exception as e:
        if status == "High_Heat_Fault":
            ai_response = "EMERGENCY: High heat detected. Increase coolant flow. (Local Override)"
        else:
            ai_response = "All systems normal. Continue monitoring. (Local Override)"

    # Step C: Safety Verification
    safety = safety_interlock(temp, press, ai_response)

    return status.replace("_", " "), ai_response, safety

---
## 4. Live Dashboard

Run this cell to launch the interactive Gradio interface directly inside your notebook.

---



In [ ]:
demo = gr.Interface(
    fn=process_monitor,
    inputs=[
        gr.Slider(300, 500, value=350, label="Reactor Temp (K)"),
        gr.Slider(50, 250, value=100, label="Pressure (psi)"),
        gr.Slider(10, 100, value=50, label="Inlet Flow (L/min)")
    ],
    outputs=[
        gr.Label(label="System Status"),
        gr.Textbox(label="AI Engineer Instruction"),
        gr.Textbox(label="Safety Interlock Status")
    ],
    title="☢️ CSTR Reactor Safety Dashboard",
    description="Agentic AI system combining Random Forest (Fault Detection) and Gemini LLM (SOP Reasoning) with a hard-coded Safety Interlock.",
    theme="glass"
)

demo.launch(debug=True)

/usr/local/lib/python3.12/dist-packages/gradio/interface.py:171: UserWarning: The parameters have been moved from the Blocks constructor to the launch() method in Gradio 6.0: theme. Please pass these parameters to launch() instead.
  super().__init__(


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://28501e0eaf98c92fec.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Would you like me to add a **Data Visualization** block to show the training data's "Danger Zone" before launching the UI?

[Integrating Gemini AI with Google Colab](https://www.youtube.com/watch?v=PytsxNfR8GI)
This video explains how to set up your environment and use API keys securely within the Google Colab ecosystem for data science projects.